# A Demonstration of the RSA Cryptosystem

The Rivest&ndash;Shamir&ndash;Adleman ([RSA](https://en.wikipedia.org/wiki/RSA_\(cryptosystem\))) cryptosystem demonstrated here combines secure key&ndash;exchange with digital signature to provide for the _confidentiality_, _integrity_ and _authenticity_ of a private message exchanged between two parties (Alice and Bob) over an insecure channel. Moreover, an adversary (Eve) listening on this channel has no practical way of learning the content of the message, nor any way of tampering with it without the receiving party knowing the message has been corrupted.

The key&ndash;exchange component of this cryptosystem provides for message confidentiality utilizing a [_key&ndash;encapsulation mechanism_](https://en.wikipedia.org/wiki/Key_encapsulation_mechanism) (or _KEM_). This differs from more traditional approaches to key negotiation that utilize a key&ndash;_agreement_ mechanism, such as those based on the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) introduced by Diffie and Hellman in their seminal 1976 paper [_New Directions in Cryptography_](https://ee.stanford.edu/~hellman/publications/24.pdf).

The digital signature component of this cryptosystem provides for message integrity and authenticity, ideas that were conceived by Diffie and Hellman in their 1976 paper, but not implemented until Rivest, Shamir and Adleman published [_A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems_](https://people.csail.mit.edu/rivest/Rsapaper.pdf) in 1978.

### A Caveat
Note that the procedure as demonstrated provides no guarantee as to the authenticity of either party; that is, that the participants Alice or Bob (or both) are who they claim to be (see [_man&ndash;in&ndash;the&ndash;middle_](https://en.wikipedia.org/wiki/Man-in-the-middle_attack) attack for elaboration). Nor does it guarantee that the shared key encrypted in the procedure cannot be decrypted by an adversary if the private key used to decrypt it is compromised at a future date (see [_forward secrecy_](https://en.wikipedia.org/wiki/Forward_secrecy) for elaboration). Protections against either vulnerability are not precluded by this implementation, but they are omitted here to minimize conceptual clutter.

## The Procedure
The following procedure, which utilizes the services of this repository's [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module, illustrates a session in which Alice signs and encrypts a private message, and then transmits the message to Bob over an insecure channel. Meanwhile, an adversary (Eve) intercepts the message, but cannot feasibly learn its content, nor tamper with it without Bob detecting the message has been corrupted.

It is assumed that Alice, Bob and Eve all possess their own copy of the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module; but that only Alice and Bob possess the signing and encryption keys produced by this module to provide the aforementioned protections.

Alice and Bob start by importing the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module, and a module [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) cipher used here is for illustrative purposes only, and in a real&ndash;world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as _3DES_ or _AES_).

In [1]:
import os
os.chdir("../src")

# Alice and Bob (and Eve?) import the rsa and sym modules required for the session
from core import rsa
from tests.core import sym

Alice begins by calling the `make_key()` function of the `rsa` module, which returns an `RSAKey` object representing an RSA keypair. She stores her keypair in `key_a` (the suffix `_a` identifies this keypair as belonging to Alice). Alice extracts her public key from the keypair and stores it in `pub_key_a`.

In [2]:
# Alice computes her keypair, and stores her public key in pub_key_a
key_a = rsa.make_key()
pub_key_a = key_a.public_key()

Alice must keep the private components of her keypair secret. These are `key_a.p` and `key_a.q` (the prime factors of the public modulus `key_a.n`), `key_a.d3` (her signing key) and `key_a.d5` (her decryption key).

Let's inspect these values to get an idea of their size and proportions.

In [3]:
key_a.p

97600269798271805239614799747261288666203347944923825880502483240536773401611726353584755482806292650927359829848800759195499257567127378882302413767378294148424443322027004018423005973113285126093372260442478967617809572347905377685552462531808922495254913830973670194565918104281878292080241115677786077953

In [4]:
key_a.q

168197384375805118806538801491139835673659643506453437188078034206407894893507016907995208806382517866453722987044537059056193198523162634077208123443394348340471679018447432350587673763913615271867149484502010841127733137241862202075653276420470801553685071583912287521744827574726489073395963673954224107007

In [5]:
key_a.n

16416110094442206349350479077629702152182650059397804829294613424415034399424506362547610026371840083389866766661466186655950817838001006499777673037089052198952963262868413503022007427036690205646296223310665826656000159949179612726092744496499824868214636077758341732125034762135467465748882015222644730023427208587781867646809490481968223384501127019506432062354054818899055475087246438442646812350768911876245215467070443802247938755008579676371681048979078367488010944529350668809501551558736278733041122033033087078154770709795275338006849741759919451019162651988236207317477149648701166500415009150478315516671

Note that `key_a.n`, which is the product of `key_a.p` and `key_a.q`, is twice the length of each individually, and that these are very big numbers.

Let's inspect the bit lengths of `key_a.n` and its prime factors.

In [6]:
key_a.modulus_size()

2048

In [7]:
key_a.p.bit_length()

1024

In [8]:
key_a.q.bit_length()

1024

Note here that a decimal number with a bit&ndash;length of 2048 (the default key size if none is specified in the `rsa.make_key()` function) has approximately 620 digits, and that a decimal number with a bit&ndash;length of 1024 has approximately 310 digits.

Here we prove that `key_a.n` is in fact the product of `key_a.p` and `key_a.q`. Also note that `pub_key_a`, which Alice fetched previously from the method `key_a.public_key()`, is in fact the same value as `key_a.n`.

In [9]:
key_a.p * key_a.q == key_a.n == pub_key_a

True

Now it's Bob's turn to generate a keypair (to differentiate Bob's keypair from Alice's, we name Bob's `key_b`).

In [10]:
# Bob computes his keypair, and stores his public key in pub_key_b
key_b = rsa.make_key()
pub_key_b = key_b.public_key()

Now that Alice and Bob have generated their keypairs, they must transmit to each other their public keys `pub_key_a` and `pub_key_b`. Once they have done this, they can proceed with the secure message exchange.

Let's say Alice wants to send the following message securely to Bob. First, Alice stores the message in `mA`.

In [11]:
# Alice's message to Bob
mA = "Sign and encrypt me!"

Then Alice calls the `key_a.sign()` method, passing it `mA`. She stores the signature in `sA`.

In [12]:
# Alice signs the message, and stores the signature in sA
sA = key_a.sign(mA)

Next, Alice calls the `rsa` module function `encrypt_key()`, passing it Bob's public key `pub_key_b`. This computes a session key that will be used to encrypt the private message using a symmetric cipher (e.g., _3DES_ or _AES_).

The function returns the pair `KA` and `cA`, which are, respectively, the symmetric encryption key, and the ciphertext of its input material; the same material Bob will use to recover the session key on his side of the channel.

In [13]:
# Alice computes a session key and stores the plaintext version KA, and the ciphertext version in cA
KA, cA = rsa.encrypt_key(pub_key_b)

Alice must keep the session key (`KA`) private.

Next, Alice encrypts the private message, calling the `encrypt()` function of the symmetric cipher. To this function she passes her symmetric key and the plaintext of her message. The function returns the ciphertext (`mAC`) of the message (`mA`).

In [14]:
# Alice encrypts the message, and stores the ciphertext in mAC
mAC = sym.encrypt(KA, mA)

If we inspect the ciphertext, we see that to Eve it would appear as a random string of bytes bearing no resemblance to the original message.

In [15]:
mAC

b'\xe9\xde{\xce3$\xea\xc1\xae\xd2\x9cA\xdf\x18Rw\xd4\x90\x84=\x13~\xeeV}Z\xa2\xa5\x8a]\xf5\xed'

Next, Alice transmits the ciphertext of the message (`mAC`), the signature of the message (`sA`), and the ciphertext of the symmetric key (`cA`) to Bob.

In order to decrypt the message, Bob must first recover the session key Alice used to encrypt it. He does this by calling  `key_b.decrypt_key()`, passing it the ciphertext of the symmetric key that Alice sent to him. Because Alice used Bob's public key (`pub_key_b`) to encrypt her session key (`KA`), Bob can recover the session key from (`cA`) using his corresponding private key.

In [16]:
# Bob recovers the session key from its ciphertext
KB = key_b.decrypt_key(cA)

Here we prove that the key Bob decrypted (`KB`), is the same as the key Alice computed (`KA`).

In [17]:
KB == KA

True

Bob calls the `decrypt()` function of the symmetric cipher, passing it the session key and the message ciphertext, to recover Alice's original plaintext. He stores the result in `mB`.

In [18]:
# Bob decrypts the message cipertext
mB = sym.decrypt(KB, mAC)

Here we prove that the recovered plaintext indeed matches Alice's original message.

In [19]:
mB == mA

True

We can inspect `mB` to see that it indeed matches Alice's original message.

In [20]:
mB

'Sign and encrypt me!'

In order to complete the session, all that remains is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, more precisely, that it was signed with Alice's private key).

Bob calls the `rsa` module function `verify()`, passing it Alice's public key `pub_key_a`, the decrypted message `mB`, and the sigature `sA`.

In [21]:
# Bob verifies the message signature
rsa.verify(pub_key_a, mB, sA)

True

If the assertion above holds, Bob can be assured that the message "Sign and encrypt me!", even if intercepted by Eve, can be read only by him and Alice; and, further, that the message was not corrupted in any way by Eve.